<a href="https://colab.research.google.com/github/aadiraju/COSC320PlagiarismDetector/blob/main/COSC320ThirdMilestone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COSC320 Third Milestone


## Rabin-Karp Function 
The following cell defines a function that implements Rabin-Karp on the given pattern and text, and returns a plagiarism percentage from 0% - 100%

In [1]:
# Following program is the python implementation of 
# Rabin Karp Algorithm given in CLRS book 
  
# pat  -> pattern 
# txt  -> text 
# q    -> A prime number 
  
def rabinKarpSearch(pat, txt): 
    q = 101 # A prime number 
    M = len(pat) 
    N = len(txt) 
    i = 0
    j = 0
    p = 0    # hash value for pattern 
    t = 0    # hash value for txt 
    h = 1
    # d is the number of characters in the input alphabet 
    d = 256
  
    # The value of h would be "pow(d, M-1)% q" 
    for i in range(M-1): 
        h = (h * d)% q 
  
    # Calculate the hash value of pattern and first window 
    # of text 
    for i in range(M): 
        p = (d * p + ord(pat[i]))% q 
        t = (d * t + ord(txt[i]))% q 
  
    # Slide the pattern over text one by one 
    for i in range(N-M + 1): 
        # Check the hash values of current window of text and 
        # pattern if the hash values match then only check 
        # for characters on by one 
        if p == t: 
            # Check for characters one by one 
            for j in range(M): 
                if txt[i + j] != pat[j]: 
                    break
  
            j+= 1
            # if p == t and pat[0...M-1] = txt[i, i + 1, ...i + M-1] 
            if j == M: 
                return M #get length of pattern that has a match
  
        # Calculate hash value for next window of text: Remove 
        # leading digit, add trailing digit 
        if i < N-M: 
            t = (d*(t-ord(txt[i])*h) + ord(txt[i + M]))% q 
  
            # We might get negative values of t, converting it to 
            # positive 
            if t < 0: 
                t = t + q
    return 0

def rabinKarpFileMatch(file, cFile):
  corpusFileSize = len(cFile)
  totalFoundChar = 0
  for line in file:
    totalFoundChar += rabinKarpSearch(line,cFile)
  return float(totalFoundChar/corpusFileSize)*100 # value from 0-100 that represents the percentage of matches found between the two documents 

# Driver program to test the above function 
txt = "COSC320 is a very fun class and I would take it again if I could"
pat = ["very fun class ","aksdjfhaskjl ","take it again "]

print("{:.2f}% similarity".format(rabinKarpFileMatch(pat, txt))) 

45.31% similarity


## KMP Function 
The following cell defines a function that implements KMP on the given pattern and text, and returns a plagiarism percentage from 0% - 100%

In [2]:
def kmpHelper(file,cFile):    
    totalFoundChar = 0
    for i in file:
      totalFoundChar += KMPSearch(i,cFile)
    
    corpusFileSize = len(cFile)
    return float(totalFoundChar/corpusFileSize)*100
    
    # for each string in data, run a for loop to check against each string in the data2 string, so that can be done, and then to return a 


def KMPSearch(pat, txt): 
    M = len(pat) 
    N = len(txt) 
    lps = [0]*M 
    j = 0
    computeLPSArray(pat, M, lps) 
    i = 0 
    while i < N: 
        if pat[j] == txt[i]: 
            i += 1
            j += 1
        if j == M: 
            return M
            j = lps[j-1] 

        elif i < N and pat[j] != txt[i]: 
            if j != 0: 
                j = lps[j-1] 
            else: 
                i += 1
    return 0
  
def computeLPSArray(pat, M, lps): 
    len = 0 # length of the previous longest prefix suffix 
  
    lps[0] # lps[0] is always 0 
    i = 1
  
    # the loop calculates lps[i] for i = 1 to M-1 
    while i < M: 
        if pat[i]== pat[len]: 
            len += 1
            lps[i] = len
            i += 1
        else: 
            # This is tricky. Consider the example. 
            # AAACAAAA and i = 7. The idea is similar  
            # to search step. 
            if len != 0: 
                len = lps[len-1] 
  
                # Also, note that we do not increment i here 
            else: 
                lps[i] = 0
                i += 1
  
txt = "ABABDABACDABABCABABABCABCBABCABCBACBACBACBACBABCBACBABD"
pat = ["CAB","BAB"]
print("{:.2f}% similarity".format(kmpHelper(pat, txt))) 



10.91% similarity


## LCSS Function 
The following cell defines a function that implements LCSS on the given pattern and text, and returns a plagiarism percentage from 0% - 100%

In [3]:
def lcs(file, cFile): 
    # find the length of the strings 
    m = len(file) 
    n = len(cFile) 
  
    # declaring the array for storing the dp values 
    L = [[None]*(n + 1) for i in range(m + 1)] 
  
    """Following steps build L[m + 1][n + 1] in bottom up fashion 
    Note: L[i][j] contains length of LCS of file[0..i-1] 
    and cFile[0..j-1]"""
    for i in range(m + 1): 
        for j in range(n + 1): 
            if i == 0 or j == 0 : 
                L[i][j] = 0
            elif file[i-1] == cFile[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1]) 
  
    # L[m][n] contains the length of LCS of file[0..n-1] & cFile[0..m-1] 
    return L[m][n] 
# end of function lcs 
def lcsHelper(file,cFile):
  totalFoundChar = 0
  for i in file:
    totalFoundChar += lcs(i,cFile)  
  corpusFileSize = len(cFile)
  return float(totalFoundChar/corpusFileSize)*100
  
# Driver program to test the above function 
file = ["AGG"]
cFile = "GXTXAYB"
print("Similarity Percentage is {:.2f}%".format(lcsHelper(file, cFile))) 

Similarity Percentage is 14.29%


## Main Driver Function
The following function will take the actual input file and compare it against each file in the corpus and return a plagiarism percentage per file.

In [20]:
import os
directory = "g17_corpusfinal/"
corpus = []
corpusNames = []
plagF = ""
dirSize = 10
counter = 0

for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".txt") and filename != 'plagarism_test_file.txt' and counter < dirSize: 
         #Take in each corpus file as a big string where newlines are replaced by spaces
         cf = open(os.path.join(directory,file), "r", encoding="utf-8")
         content = cf.read()
         content_list = content.replace('\n',' ')
         corpus.append(content_list)
         corpusNames.append(filename)
         counter+=1
         cf.close()  
     if filename == 'plagarism_test_file.txt':
        # For the plagirarism file, split up by line, in a list.
        with open(os.path.join(directory,file),'r', encoding="utf-8") as plagFile:
            plagF = [line.rstrip('\n') for line in plagFile]

In [22]:
corpus[0]

' In probability theory, Bayes\' theorem also called Bayes\' law after Rev Thomas Bayes compares the conditional and marginal probabilities of two random events. It is often used to calculate posterior probabilities given observations. For example, a patient may be observed to have certain symptoms. Bayes\' theorem can be used to calculate the likelihood that a proposed analysis is accurate, given that observation.  As an official theorem, Bayes\' theorem is valid in all universal interpretations of probability. However, it plays a fundamental role in the debate around the foundations of statistics: frequentist and Bayesian interpretations disagree about the ways in which probabilities should be assigned in applications.  Frequentists assign probabilities to random events according to their frequencies of happening or to subsets of populations as proportions of the whole. Whilst Bayesians describe probabilities in terms of beliefs and degrees of uncertainty. The articles on Bayesian pr

In [ ]:
#Take in each corpus file as a big string where newlines are replaced by spaces
cf = open(cfile, "r")
content = cf.read()
content_list = content.replace('\n',' ')
cf.close()  
    
# For the plagirarism file, split up by line, in a list.
with open(file,'r') as plagFile:
    data2 = plagFile.readLines().replace('\n',' ')